<a href="https://colab.research.google.com/github/absolutelydawn/TestNeoX/blob/main/chatbotTest01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall transformers -y
!pip install transformers

Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)


In [6]:
!pip install transformers --upgrade
from transformers import GPTNeoTokenizer

ImportError: ignored

In [2]:
!pip install accelerate

In [7]:
!pip install transformers
import pandas as pd
import re
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, GPT2Config, Trainer, TrainingArguments, DataCollatorForLanguageModeling


# 데이터 불러오기
with open("ChatbotData.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

input_texts = []
output_texts = []
for line in lines:
    input_text, output_text, _ = line.split("\t")
    input_texts.append(re.sub("[^가-힣a-zA-Z?.!,]+", " ", input_text).strip())
    output_texts.append(re.sub("[^가-힣a-zA-Z?.!,]+", " ", output_text).strip())

# 토큰화
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")  # GPT-Neo 모델의 토크나이저인 GPT2Tokenizer를 사용
input_encodings = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
output_encodings = tokenizer(output_texts, return_tensors="pt", padding=True, truncation=True)

# 모델 설정
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")  # GPT-Neo 모델을 로드


# 모델 학습
training_args = TrainingArguments(
    output_dir="./gpt_neo_chatbot_model",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    deepspeed="ds_config.json",  # deepspeed를 사용하려면 해당 설정 파일이 필요합니다.
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=input_encodings,
    data_collator=data_collator,
)

trainer.train()

# 학습된 모델 저장
trainer.save_model("./gpt_neo_chatbot_model")


FileNotFoundError: ignored